Pletnev Aleksandr

In [71]:
import nltk
from random import randint
from nltk.lm.models import Laplace
import numpy as np
import torch
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import functional
import torch.nn as nn
import math
import json

from torch.autograd import Variable
from torch.nn import functional
import torch.nn as nn
import math
from collections import Counter

from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends, flatten

## Problem 3.1

### 1. Implement a function that generates a dinosaur name of random length. The generated dinosaur name should start with the begin symbol < and with the end symbol >.

In [499]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]

In [500]:
# Step 1. Compute frequency of each character
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

In [501]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [502]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

In [503]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [504]:
pred_symbol = '<'
name = "<"
# Stop generating names when we predict final symbol
for x in  list(range(1,100)):
    pred_symbol = cprob[pred_symbol].generate()
    name+=pred_symbol
    if (pred_symbol == '>'):
        break

In [505]:
print("Generated name: ", name)

Generated name:  <donyatoruaeliauirurusangocoriloibos>


In [506]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


### 2. Implement and apply add-one smoothing. 

In [550]:
vocabulary = nltk.FreqDist(flatten(names))

In [551]:
# add Laplace (add-one) smothing
cprob_laplace = nltk.ConditionalProbDist(cfreq, nltk.LaplaceProbDist, bins=len(vocabulary))

In [552]:
pred_symbol = '<'
name = "<"
# Stop generating names when we predict final symbol
for x in  list(range(1,100)):
    pred_symbol = cprob_laplace[pred_symbol].generate()
    name+=pred_symbol
    if (pred_symbol == '>'):
        break
print("Generated name: ", name)

Generated name:  <eroiauranglolitomonaong>


In [553]:
print('p(a a) = %1.4f' %cprob_laplace['a'].prob('a'))
print('p(a b) = %1.4f' %cprob_laplace['a'].prob('b'))
print('p(a u) = %1.4f' %cprob_laplace['a'].prob('u'))

p(a a) = 0.0048
p(a b) = 0.0099
p(a u) = 0.3149


In [554]:
print('p(y y) = %1.4f' %cprob['y'].prob('y'))
print('p(y y) after smoothing = %1.4f' %cprob_laplace['y'].prob('y'))

p(y y) = 0.0000
p(y y) after smoothing = 0.0034


How add one smoothing affects the model and the generated names?
- It is hard notice the trend of how it affects the generated names. But it is clearly, that now rare seldom bigrams occur more often and frequent bigram occur less often
- If some bigram had 0 probability, now it have at least prob -> names can be generated more variously. On other hand, it allows names to include some unreal combinations of words, for example: yy

## Problem 3.2

In [6]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

In [1021]:
class NLM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        
        #One hidden dense layer
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.2)
        # Output layer
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        #There are as many units in the output layer, as there are symbols in the dictionar
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        
        # Softmax performs terrible (it just stucks) . LogSoftMax is much better
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, h_prev, x):
#print(h_prev.shape)
 #       print(x.shape)
        input_combined = torch.cat([h_prev, x], dim = 1)
        hidden = torch.tanh(self.dropout(self.i2h(input_combined)))
        
        output = self.i2o(input_combined)
        
        
        #№output_combined = torch.cat((hidden, output), 1)
        #output = self.o2o(output_combined)
        #output = self.dropout(output)
        output = self.softmax(output)
        return hidden, output

In [1022]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [1023]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

In [1336]:
def train_one_epoch_nlm(model, loss_fn, optimizer,dataset = trn_dl):
    model.train()
    # X - correct input - onehot
    # y - onehot
    for line_num, (x, y) in enumerate(dataset):
        loss = 0
        optimizer.zero_grad()
        
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]

        x[0, start_char_idx] = 1
        word_size=0
        predicted_char_idx = start_char_idx
        # As in instructions - step read k characters, predict next one
        for i in range(1,y.shape[1]):
            ## 
            #THE INPUT OF THE MODEL ARE CONCATENATED ONE-HOT ENCODINGS OF CHARACTERS
            # MaX - 870 = 29 * 30 (max length of word)
            
            # Generate required tensor
            xxx = torch.zeros([len(indices), trn_ds.vocab_size], dtype=torch.float)
            for idx in range(0,len(indices)):
                xxx[idx][indices[idx]] = 1
            #THE INPUT OF THE MODEL ARE CONCATENATED ONE-HOT ENCODINGS OF CHARACTERS
            xx = torch.cat([xxx[:word_size+1].view(-1), torch.zeros( 870-xxx[:word_size+1].view(-1).shape[0] )], 0)
            h_prev, y_pred = model(h_prev, xx[None])
                        # as in instructions - softmax
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.detach().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            loss += loss_fn(y_pred, y[:, i])
            word_size += 1
        if (line_num+1) % 100 == 0:
            # Create test output
            print_sample(sample_nlm(model))
        loss.backward()
        optimizer.step()

In [1337]:
def train_nlm(model, loss_fn, optimizer, dataset=trn_dl, epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch_nlm(model, loss_fn, optimizer,dataset)
        print()

In [1271]:
def sample_nlm(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]

        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        # As in instructios - input - onehot encoded, predict next symbol!
        while predicted_char_idx != newline_idx and word_size != 29:
            #print(sum(x[:, :(word_size+1)]).shape)
            
            # Generate required tensor
            x = torch.zeros([len(indices), trn_ds.vocab_size], dtype=torch.float)
            for idx in range(0,len(indices)):
                x[idx][indices[idx]] = 1

            #THE INPUT OF THE MODEL ARE CONCATENATED ONE-HOT ENCODINGS OF CHARACTERS
            xx = torch.cat([x[:word_size+1].view(-1), torch.zeros( 870-x[:word_size+1].view(-1).shape[0] )], 0)

            h_prev, y_pred = model(h_prev, xx[None])
            
            
            # as in instructions - softmax
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.detach().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            predicted_char_idx = idx
            word_size += 1
        
        if word_size == 29:
            indices.append(newline_idx)
    return indices

In [1272]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 150 #50 #1000 #100
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [1273]:
nlm = NLM(trn_ds.vocab_size*30, hidden_size, trn_ds.vocab_size).to(device)

In [1274]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(nlm.parameters(), lr=1e-2)

In [1275]:
train_nlm(nlm, loss_fn,optimizer,epochs = 20)

Epoch:1
<uoareuooaxar>
<neulnauas>
<dtilauar>
<l<totu>
<errl<tnrtuaooaluaum>
<ilahsasn>
<alwpucuuusr>
<smerrn>
<asssoitus>
<rstanoaoue>
<gsarhtacsrasn>
<snandunpauas>
<smanetnoauas>
<uailahsaur>
<hasalvrydu>

Epoch:2
<lerrmyusux>
<shtaasoargursa>
<iwouatuusr>
<rrlysnrsuarsas>
<iysmpsuarsas>
<wouasttrouuu>
<lamdsilauas>
<atiiataq>
<acrnanauosa>
<aritacsoaso>
<sngrro>
<arsrnestr>
<rrtaknamuaxr>
<kancsinaucs>
<trnersn>

Epoch:3
<uslerso>
<ouastusouuu>
<uaasoareussa>
<inaisauouar>
<qrtanoanua>
<ldsiiatar>
<otasssohtus>
<gtacroarlursg>
<uaimahsauo>
<taluouauuuss>
<sariuaausaus>
<esariuaausaus>
<irrtanoaoue>
<igrrnytruu>
<yslortaorar>

Epoch:4
<rtalnagtaur>
<uaasnaoaursd>
<iyslpsuarsas>
<unucsssoiuus>
<iysnrsuarsas>
<eiasan>
<irstanrar>
<rrkysosu>
<uaknaiuaur>
<asakuouauuuts>
<andsiiauas>
<iamdsilauas>
<kanesilauas>
<rrkysnsuyasses>
<iesiiatar>

Epoch:5
<iahsasnuanusaur>
<iysnpstarsas>
<rtalnahtaur>
<acrnaobuora>
<ashiasao>
<uacsoareurse>
<rmerro>
<allahsaup>
<rnersnzuruu>
<hkasanusun>
<esas

It can produce names, which cont-based  language can't produce (if we don't apply smoothing). Therefore - much more variabity  in generated names. Also we can indefinetely train the model until overfitting, but it generalizes well
Also the quality of generated words is not as good and speed is of training is quite slow. I even had to decrease max length of word so training time would be reasonable
It also needs much more epochs to produce okay names


Parameters: size of hidden layer, max word size, dropout percentage, epochs, learning rate. 
- Max word size: after changing didn't change much, because words rarely reach >20 symbols. But the training time depends on it
- size of hidden layer - increase of computation time, but quality and perplexity of names change a bit much  if increase (except if we set very small hidden layer). It has to big for proper generation
- dropout - increase of computation time, but quality of names didn't change much (except if we set very high percentage
- epochs & learning reate - epochs should be > 10 to generate proper names

## Problem 3.3

In [1416]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
 
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

def train_one_epoch(model, loss_fn, optimizer,dataset = trn_dl):
    model.train()
    for line_num, (x, y) in enumerate(dataset):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()
        
def train(model, loss_fn, optimizer, dataset=trn_dl, epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn,optimizer, dataset)
        print()

In [1042]:
# The RNN from class practise
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(combined)
        return h, y

In [1043]:
loss_fn = nn.CrossEntropyLoss()
rnnn = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
optimizer = optim.SGD(rnnn.parameters(), lr=1e-2)
train(rnnn, loss_fn, optimizer, epochs = 20)

Epoch:1
<naiua>
<hwacsoarts>
<jatas>
<qveus>
<merus>
<unrusaoras>
<auluaauras>
<uirasansus>
<tursr>
<snyus>
<suaosaisas>
<valusaopus>
<auaos>
<werusdirus>
<torusanras>

Epoch:2
<aumuc>
<sraop>
<hlaudourus>
<turlsuutus>
<suainarras>
<uaeusaurus>
<aubls>
<veruteorus>
<trrusairas>
<aurucnuras>
<smrasapsus>
<surosturus>
<rusasoarur>
<fuaeshcrus>
<hbras>

Epoch:3
<pueus>
<lirus>
<sorusamras>
<aumubauras>
<snrasausus>
<surortur>
<sprusaurhs>
<atruaauras>
<snrasausus>
<suros>
<rpuus>
<suaosaesis>
<tbgusaurus>
<aucnsturus>
<llrus>

Epoch:4
<srrusauras>
<aupucauras>
<snrasassus>
<suronturus>
<puranraryc>
<euacsicrus>
<hcras>
<pueusuurus>
<zurostcrur>
<scrosairua>
<curoaterus>
<burusaurus>
<luruaeueussur>
<ahroalnrus>
<tdivsony>

Epoch:5
<spcpsotontotaurus>
<aupucauraurus>
<aublsiurus>
<llrus>
<sprusauras>
<aurucauras>
<snrasausus>
<surnltur>
<sprutaurhs>
<aurubauras>
<snrasassus>
<surorturus>
<purasaerur>
<fuaasibrus>
<hcras>

Epoch:6
<pueus>
<llrus>
<sorusaoras>
<aurucaurhs>
<smrasausus>
<suro

In [1045]:
print_sample((sample(rnnn)))

<llsonzurus>


Key differences:
- RNN trains faster because the input layer is not as big
- RNN performs better, words are more real
- it doesn't remember all prev symbols in constrast with NLP
- It is similar to the NLM, just that after encoding the current input word we feed the resulting representation into a LSTM, which then outputs a vector. Then we use the decoder to convert this output vector into a vector of probability values

Parameters:
- NLM: 5 parameters size of hidden layer, max word size, dropout percentage, epochs, learning rate.
- Recurrent: also 5 parameters, but max_word_size is not such important

## Problem 3.4.A

In [1295]:
#calculate sampe prob for RNN model
def sample_probablity(name,model):
    model.eval()
    total_probab = 1.0
    h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)

    x = h_prev.new_zeros([1, trn_ds.vocab_size])

    start_char_idx = trn_ds.ch_to_idx['<']

    indices = [start_char_idx]

    x[0, start_char_idx] = 1
    predicted_char_idx = start_char_idx
    # One-code input name
    x = torch.zeros([len(name), trn_ds.vocab_size], dtype=torch.float)
    for idx in range(0,len(name)):
        x[idx][trn_ds.ch_to_idx[name[idx]]] = 1
    # Calculate prob (just by taking maximum in y_softmax_scores)
    for idx in range(0,len(name)-1):
        h_prev, y_pred = model(h_prev, x[idx][None])
        y_softmax_scores = torch.softmax(y_pred, dim=1)
        total_probab*=y_softmax_scores.max().item()
    return total_probab

In [1296]:
#calculate sample prob for nlm model
def sample_probablity_nlm(name,model):
    model.eval()
    total_probab = 1.0
    h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)

    x = h_prev.new_zeros([1, trn_ds.vocab_size])

    start_char_idx = trn_ds.ch_to_idx['<']

    indices = [start_char_idx]
    
    x[0, start_char_idx] = 1
    predicted_char_idx = start_char_idx
    # One-code input name
    x = torch.zeros([len(name), trn_ds.vocab_size], dtype=torch.float)
    for idx in range(0,len(name)):
        x[idx][trn_ds.ch_to_idx[name[idx]]] = 1
    # Calculate prob (just by taking maximum in y_softmax_scores)
    for idx in range(0,len(name)-1):
        
        xx = torch.cat([x[:idx+1].view(-1), torch.zeros( 870-x[:idx+1].view(-1).shape[0] )], 0)
        h_prev, y_pred = model(h_prev, xx[None])
        
        y_softmax_scores = torch.softmax(y_pred, dim=1)
        total_probab*=y_softmax_scores.max().item()
    return total_probab

In [1297]:
def get_prob(name,lm):
    """
    lm = count-based LM,
    or
    count-based with add-one smoothing LM
    or
    neural LM
    or
    RNN-LM
    """
    total_prob = 1.0
    if lm == 'count-based LM':
        for idx in range(0,len(name)-1):
            total_prob *= cprob[name[idx]].prob(name[idx+1])
    if lm == 'count-based with add-one smoothing LM':
        for idx in range(0,len(name)-1):
            total_prob *= cprob_laplace[name[idx]].prob(name[idx+1])
    if lm == 'neural LM':
        total_prob = sample_probablity_nlm(name,nlm)
    if lm == 'RNN-LM':
        total_prob = sample_probablity(name,rnnn)

    return total_prob

In [1303]:
name = '<aukucauras>'

In [1304]:
get_prob(name,'count-based LM')

4.464050746210654e-11

In [1305]:
get_prob(name,'count-based with add-one smoothing LM')

3.804742604109511e-11

In [1306]:
get_prob(name,'neural LM')

3.4510260682161353e-09

In [1307]:
get_prob(name,'RNN-LM')

4.915068917332468e-06

## 3.4.B

In [1408]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [1409]:
from sklearn.model_selection import train_test_split 
train, test = train_test_split(trn_ds.lines,test_size = 0.33)

In [1410]:
trn_ds_train = DinosDataset()
trn_ds_train.lines = train
trn_dl_train =  DataLoader(trn_ds_train, shuffle=True)
trn_ds_test = DinosDataset()
trn_ds_test.lines = test
trn_dl_test =  DataLoader(trn_ds_test, shuffle=True)

In [1411]:
## count-based LM 
names = train
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)
l = sum([freq[char] for char in freq])
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [1412]:
## add-one smoothing LM
vocabulary = nltk.FreqDist(flatten(names))
cprob_laplace = nltk.ConditionalProbDist(cfreq, nltk.LaplaceProbDist, bins=len(vocabulary))

In [1413]:
# neural LM
nlm = NLM(trn_ds.vocab_size*30, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(nlm.parameters(), lr=1e-2)
train_nlm(nlm, loss_fn,optimizer,epochs = 10,dataset = trn_dl_train)

Epoch:1
<oaluauo>
<sm>
<alypubuvusn>
<rlzsmrsuanoanuauo>
<uilauas>
<sngrro>
<astsnesso>
<asnaoeurra>
<lerrm>
<rrjysmosuarscs>

Epoch:2
<ashtaatsasr>
<lcsiiasao>
<errnytpsu>
<atalupuatuusr>
<csiiatao>
<atalusuauu>
<almagtauo>
<kahsasi>
<xoucstsohuus>
<rlysnrtuassas>

Epoch:3
<asitadtsaus>
<siiatao>
<rlysnrtuarsas>
<ilahsasmyai>
<hasalusuduu>
<uainaltaur>
<uainaiuaur>
<oucstsoiuuu>
<hasanusueu>
<iahrasiuai>

Epoch:4
<aluotcsuusr>
<asssnestr>
<uainahuaur>
<arhtaasparo>
<snprtanoap>
<gjatalurueu>
<rlysmrsuassds>
<imaisatn>
<ahsashuacusasr>
<ataluouauuysr>

Epoch:5
<amdsiiauas>
<strngrsr>
<iamesilauas>
<imaisatn>
<fiasanusyeu>
<tsngrrn>
<iepplysosu>
<rlysnrsuarses>
<acroaoeurra>
<acsnaoduora>

Epoch:6
<ancsiiauas>
<uainahuaus>
<yslpsuaosas>
<smpstanpapua>
<iwouasuurox>
<uacsnaoeuora>
<ialesimauas>
<rlaneslnaucs>
<ipstanoaoua>
<rnandskmauas>

Epoch:7
<iahrasiuaiusaus>
<smrstaorar>
<ivotcsuurou>
<ucsssogstr>
<astsngstr>
<rriysnsuuasuos>
<asmanetmoauis>
<sasiuaausatr>
<uaasnaoaurse>
<acrnaoauo

In [1417]:
# RNN_LM 
loss_fn = nn.CrossEntropyLoss()
rnnn = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
optimizer = optim.SGD(rnnn.parameters(), lr=1e-2)
train(rnnn, loss_fn, optimizer, epochs = 10,dataset = trn_dl_train)

Epoch:1
<maiuauo>
<actoaofsosauaoyutau>
<uniustysus>
<waotatuc>
<huaes<arts>
<iaras>
<pueus>
<merus>
<tnrusaorgs>
<aulucctras>

Epoch:2
<tinasausus>
<turus>
<sayrss>
<uaroceugus>
<aaurasaurug>
<antusdusus>
<stoysosrua>
<laluh>
<guabslaros>
<hasas>

Epoch:3
<pugus>
<lirus>
<tauruciueusaurun>
<saarns>
<haueus>
<verusoisus>
<trrusauros>
<auouaaurasouroc>
<antssaus>
<lauruv>

Epoch:4
<liuras>
<aaudus>
<agroaterusaulus>
<veruseurus>
<snrusaurns>
<auruaauras>
<tnrasaisus>
<sourisuusus>
<suaosauros>
<uaerucuisusoulus>

Epoch:5
<vaurus>
<siusus>
<srbrocaueus>
<agrobanrus>
<tcrusvurus>
<dtrov>
<lurucauius>
<saucessarus>
<hcsas>
<pugusturus>

Epoch:6
<zlrrusaoras>
<ainsantuaus>
<gnbuchurup>
<turistutus>
<suaorasros>
<tacrualurus>
<tarsusausus>
<strysiuruc>
<lcctaorybdosaurus>
<aucottun>

Epoch:7
<tmurus>
<zurus>
<anras>
<auruaauras>
<snrasaisuuaus>
<llrur>
<srrusauras>
<auquaauras>
<snrasausus>
<smuros>

Epoch:8
<kyurus>
<anrar>
<auruaauras>
<tatcumous>
<burusdus>
<zurustcrus>
<saurucauris>
<sam

## 3.4. C

In [1419]:
entropy_count_based = 0 
entropy_smoothing_LM =0 
entropy_nlm = 0
entropy_rnn = 0
for name in test:
    entropy_count_based+=get_prob(name,'count-based LM')
    entropy_smoothing_LM+=get_prob(name,'count-based with add-one smoothing LM')
    entropy_nlm+=get_prob(name,'neural LM')
    entropy_rnn+=get_prob(name,'RNN-LM')
entropy_count_based /=len(test)
entropy_smoothing_LM /=len(test) 
entropy_nlm /=len(test)
entropy_rnn /=len(test)

In [1424]:
# Perplexity:
print("Perplexity for count-based LM ", 2**entropy_count_based)
print("Perplexity for count-based LM with smoothing ", 2**entropy_smoothing_LM)
print("Perplexity for NLM", 2**entropy_nlm)
print("Perplexity for RNN-LM ", 2**entropy_rnn)

Perplexity for count-based LM  1.0000003375260127
Perplexity for count-based LM with smoothing  1.0000003003758293
Perplexity for NLM 1.0000012759394068
Perplexity for RNN-LM  1.0006227088755617
